In [2]:
import os
import pandas as pd

def load_excel_sheet(file_path):
    """
    Lädt das Arbeitsblatt 'Transaktionen zum Indiztreffer' aus der angegebenen Excel-Datei.
    
    Parameter:
      file_path (str): Pfad zur Excel-Datei.
      
    Rückgabe:
      DataFrame mit den geladenen Daten oder None, falls ein Fehler auftritt.
    """
    try:
        df = pd.read_excel(file_path, sheet_name='Transaktionen zum Indiztreffer')
        return df
    except Exception as e:
        print(f"Die Datei {file_path} kann nicht geladen werden: {e}")
        return None

def load_all_files(directory_path):
    """
    Lädt alle Excel-Dateien im Verzeichnis und verarbeitet diese:
      - Entfernt leere Zeilen (wenn alle Spalten leer sind),
      - Setzt den Index zurück,
      - Extrahiert einen 'Indikator' aus dem Dateinamen und fügt diesen als neue Spalte hinzu.
      
    Parameter:
      directory_path (str): Pfad zum Verzeichnis mit den Excel-Dateien.
      
    Rückgabe:
      Dictionary {Dateiname: DataFrame} für alle erfolgreich geladenen Dateien.
    """
    dataframes = {}
    for filename in os.listdir(directory_path):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(directory_path, filename)
            df = load_excel_sheet(file_path)
            if df is not None:
                # Entfernt Zeilen, in denen alle Spalten leer sind, und setzt den Index zurück
                df.dropna(how='all', inplace=True)
                df.reset_index(drop=True, inplace=True)
                
                # Extrahiert den Indikator aus dem Dateinamen
                indikator = filename.split('_')[-1].replace('.xlsx', '')
                df['Indikator'] = indikator
                dataframes[filename] = df
    return dataframes

def combine_dataframes(dataframes):
    """
    Kombiniert alle einzelnen DataFrames zu einem einzelnen DataFrame.
    
    Parameter:
      dataframes (dict): Dictionary mit einzelnen DataFrames.
      
    Rückgabe:
      kombinierter DataFrame (pd.concat).
    """
    combined_df = pd.concat(dataframes.values(), ignore_index=True)
    return combined_df

def aggregate_indikators(combined_df):
    """
    Gruppiert den kombinierten DataFrame anhand der Schlüsselspalten 
    'Kontonr.', 'Kundennr.', 'Transaktionstyp' und 'Buchungstag'. 
    Für die Spalte 'Indikator' werden alle eindeutigen Werte sortiert und zu
    einem String zusammengeführt.
    
    Parameter:
      combined_df (DataFrame): Der kombinierte DataFrame aller Dateien.
      
    Rückgabe:
      aggregierter DataFrame mit den zusammengeführten Indikatoren.
    """
    grouped_df = combined_df.groupby(
        ['Kontonr.', 'Kundennr.', 'Transaktionstyp', 'Buchungstag'], as_index=False
    ).agg({
        'Indikator': lambda x: ', '.join(sorted(set(x)))
    })
    return grouped_df

def get_final_dataframe(combined_df, grouped_df):
    """
    Erzeugt den finalen DataFrame, indem zunächst Duplikate in den 
    Schlüsselspalten entfernt werden und dann mit den aggregierten 
    Indikator-Daten zusammengeführt wird.
    
    Parameter:
      combined_df (DataFrame): Der kombinierte DataFrame aller geladenen Datei.
      grouped_df (DataFrame): Der aggregierte DataFrame mit den zusammengeführten Indikatoren.
      
    Rückgabe:
      finaler DataFrame, der alle Originalspalten und die aggregierten Indikator-Daten enthält.
    """
    # Entferne Duplikate basierend auf den Schlüsselspalten
    final_df = combined_df.drop_duplicates(subset=['Kontonr.', 'Kundennr.', 'Transaktionstyp', 'Buchungstag'])
    
    # Zusammenführen der aggregierten Indikator-Daten mit dem finalen DataFrame
    final_df = final_df.merge(
        grouped_df[['Kontonr.', 'Kundennr.', 'Transaktionstyp', 'Buchungstag', 'Indikator']],
        on=['Kontonr.', 'Kundennr.', 'Transaktionstyp', 'Buchungstag'],
        how='left'
    )
    
    # Falls nach dem Merge doppelte Spalten entstanden sind, die nicht benötigt werden:
    final_df.rename(columns={'Indikator_y': 'Indikator'}, inplace=True)
    if 'Indikator_x' in final_df.columns:
        final_df.drop(columns=['Indikator_x'], inplace=True)
    
    return final_df

def save_final_results(final_df, output_file_path):
    """
    Speichert den finalen DataFrame in eine Excel-Datei.
    
    Parameter:
      final_df (DataFrame): Der zu speichernde DataFrame.
      output_file_path (str): Pfad zur Ausgabedatei.
    """
    final_df.to_excel(output_file_path, index=False)
    print(f"Die Ergebnisse wurden in '{output_file_path}' gespeichert.")

def main():
    """
    Hauptfunktion:
      - Lädt alle Excel-Dateien aus dem Verzeichnis 'Indikatoren'.
      - Kombiniert diese zu einem DataFrame.
      - Aggregiert die 'Indikator'-Spalte anhand der Schlüsselspalten.
      - Erzeugt einen finalen DataFrame und speichert das Ergebnis in 'Ergebnisse.xlsx'.
    """
    directory_path = 'Indikatoren'
    output_file_path = 'Ergebnisse.xlsx'
    
    # Lade alle Excel-Dateien im angegebenen Verzeichnis
    dataframes = load_all_files(directory_path)
    if not dataframes:
        print("Keine Daten gefunden!")
        return
    
    # Kombiniere alle einzelnen DataFrames
    combined_df = combine_dataframes(dataframes)
    
    # Aggregiere die Indikatoren für identische Einträge
    grouped_df = aggregate_indikators(combined_df)
    
    # Erzeuge den finalen DataFrame, der auch alle Originalspalten enthält
    final_df = get_final_dataframe(combined_df, grouped_df)
    
    # Speichere das Ergebnis in einer neuen Excel-Datei
    save_final_results(final_df, output_file_path)

if __name__ == "__main__":
    main()

Die Ergebnisse wurden in 'Ergebnisse.xlsx' gespeichert.
